In [1]:
#from google.colab import drive
import pandas as pd
import numpy as np
#drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!pip install vncorenlp
!pip install py_vncorenlp
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645933 sha256=5000973c5616d0898bd57b4602434c7934e8016312ef2345609f529860ccc822
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.4 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=ee8ac076feeab1d137ce8c3071b3195df56fa03778c8fd094400832b25e13907
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 23.3 MB/s eta 0:00:00


In [3]:
import py_vncorenlp
from pyvi import ViPosTagger
py_vncorenlp.download_model()
client = py_vncorenlp.VnCoreNLP(annotators=["pos"])

In [4]:
test_gold = open('data/test_gold.txt', encoding='utf-8').readlines()
test_words = open('data/test_words.txt', 'r', encoding='utf-8').readlines()

In [5]:
from sklearn.metrics import classification_report
#client = py_vncorenlp.VnCoreNLP(annotators=["pos"])
def report(client):
    y_pred = []
    y_true = []
    for word in test_words:
        word = word.replace('\n', '')
        if '_' not in word:
          predict = client.annotate_text(word)
        else:
          predict = client.annotate_text(word.replace('_', ' '))
        if predict != {}:
          y_pred.append(predict[0][0]["posTag"])
        else:
          continue

    for word in test_gold:
        word_tag_tuple = word.split()
        if len(word_tag_tuple) != 2:
          continue
        word, tag = word_tag_tuple
        y_true.append(tag)
    print(classification_report(y_true, y_pred, zero_division=0))
    return y_pred, y_true

print('Kết quả khi sử dụng thư viện VnCoreNLP trên tập test:\n')
y_pred, y_true_test = report(client)

Kết quả khi sử dụng thư viện VnCoreNLP trên tập test:

              precision    recall  f1-score   support

           A       1.00      1.00      1.00        33
           C       1.00      0.64      0.78        11
          CH       1.00      1.00      1.00        45
          Cc       1.00      1.00      1.00         9
           E       0.89      1.00      0.94        32
           L       1.00      1.00      1.00        12
           M       1.00      1.00      1.00        13
           N       0.98      0.95      0.97       133
          Np       0.74      1.00      0.85        25
          Nu       0.00      0.00      0.00         1
          Ny       1.00      0.43      0.60         7
           P       0.96      1.00      0.98        22
           R       0.96      0.96      0.96        24
           T       1.00      1.00      1.00         2
           V       0.99      0.99      0.99        96
           X       1.00      0.83      0.91         6
           Z       1.00   

In [6]:
client_2 = ViPosTagger
def report(client):
    y_pred = []
    y_true = []
    for word in test_words:
        word = word.replace('\n', '')
        predict = client_2.postagging(word)
        if predict[1][0] == 'F':
          predict[1][0] = 'CH'
        if predict[1][0] == 'C':
          predict[1][0] = 'Cc'
        if predict[1][0] == 'S':
          predict[1][0] = 'C'
        if word != "":
          y_pred.append(predict[1][0])
        else:
          continue

    for word in test_gold:
        word_tag_tuple = word.split()
        if len(word_tag_tuple) != 2:
          continue
        word, tag = word_tag_tuple
        y_true.append(tag)
    print(classification_report(y_true, y_pred, zero_division=0))
    return y_pred, y_true

print('Kết quả khi sử dụng thư viện Pyvi trên tập test:\n')
y_pred, y_true_test = report(client_2)

Kết quả khi sử dụng thư viện Pyvi trên tập test:

              precision    recall  f1-score   support

           A       0.72      0.94      0.82        33
           C       0.00      0.00      0.00        11
          CH       1.00      1.00      1.00        45
          Cc       0.64      1.00      0.78         9
           E       1.00      0.03      0.06        32
           L       1.00      1.00      1.00        12
           M       1.00      0.77      0.87        13
           N       0.73      0.97      0.83       133
          Np       0.57      0.80      0.67        25
          Nu       0.00      0.00      0.00         1
          Ny       1.00      0.29      0.44         7
           P       1.00      0.09      0.17        22
           R       0.71      1.00      0.83        24
           T       0.00      0.00      0.00         2
           V       0.87      0.88      0.87        96
           X       0.00      0.00      0.00         6
           Z       0.00      0.